# AIN311: Fundamentals of Machine Learning Project (Fall 2024)
### Abdulkadir Parlak - 2210765025
### Talha Kaba - 2210765037

## Necessary Imports

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) == 0:
    raise RuntimeError("No GPU detected. Please enable GPU in the Kaggle environment.")

In [ ]:
# Paths
base_dir = "/kaggle/input/punch-detection-data-frames-scaled-180x180/Consolidated_Scaled_Data"
scaled_images_folder = "scaled_images"
scaled_annotations_file = "scaled_annotations.json"

In [ ]:
# Initialize lists for data and labels
images = []
labels = []

# Load the JSON annotations
def parse_annotations(json_file):
    with open(json_file, "r") as f:
        data = json.load(f)

    annotations = {}
    for track in data:
        for item in track.get("tracks", []):
            label = item.get("label")
            if label:
                for shape in item.get("shapes", []):
                    frame = shape.get("frame")
                    if frame is not None:
                        annotations[f"{frame}_cropped.jpg"] = label
    return annotations

In [ ]:
# Process each task folder
for task_dir in os.listdir(base_dir):
    task_path = os.path.join(base_dir, task_dir)

    if os.path.isdir(task_path):
        image_dir = os.path.join(task_path, scaled_images_folder)
        annotation_path = os.path.join(task_path, scaled_annotations_file)

        # Debugging: Check if directories and files exist
        if not os.path.exists(image_dir):
            print(f"Image directory not found: {image_dir}")
            continue
        if not os.path.exists(annotation_path):
            print(f"Annotation file not found: {annotation_path}")
            continue

        # Parse annotations
        frame_annotations = parse_annotations(annotation_path)

        # Load images and corresponding labels
        for image_name, label in frame_annotations.items():
            image_path = os.path.join(image_dir, image_name)
            if os.path.exists(image_path):
                # Load and preprocess image
                try:
                    img = load_img(image_path, target_size=(180, 180))
                    img_array = img_to_array(img) / 255.0  # Normalize pixel values
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            else:
                print(f"Image file not found: {image_path}")

In [ ]:
# Map labels to integers for classification
classes = [
    "Head with left hand", "Head with right hand",
    "Body with left hand", "Body with right hand",
    "Block with left hand", "Block with right hand",
    "Miss with left hand", "Miss with right hand"
]
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
labels = [class_to_index[label] for label in labels]

# Debugging: Check if images and labels were loaded
print(f"Total images loaded: {len(images)}")
print(f"Total labels loaded: {len(labels)}")

# Ensure data is available
if len(images) == 0 or len(labels) == 0:
    raise ValueError("No images or labels were loaded. Please check the dataset structure and file paths.")

# Convert to numpy arrays
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

In [ ]:
# Split data into training (80%) and validation (20%)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

## CNN Model

In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=90, kernel_size=(3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=180, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=180, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=180, activation='relu'),
    tf.keras.layers.Dense(units=8, activation='softmax')  # Multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training the model

In [ ]:
# Train the model explicitly on GPU with epochs=10 and batch_size=32
with tf.device('/GPU:0'):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=32
    )

## Evaluate the Model

In [ ]:
# Evaluate the model
with tf.device('/GPU:0'):

    # Calculate accuracy and final loss
    loss, accuracy = model.evaluate(X_val, y_val)
    print(f"Final Validation Loss: {loss:.4f}")
    print(f"Final Validation Accuracy: {accuracy:.4f}")

    # Calculate other evaluation metrics
    y_pred_probs = model.predict(X_val)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Display classification report
    print("Classification Report:")
    print(classification_report(y_val, y_pred, target_names=classes))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_val, y_pred)

    # Display confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()